<a href="https://www.kaggle.com/code/aisuko/optimizing-model-parameters?scriptVersionId=161402355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

We build our own model in [Create simple model using PyTorch](https://www.kaggle.com/code/aisuko/create-simple-model-using-pytorch), and we preared data in [Datasets and Dataloaders in PyTorch](https://www.kaggle.com/code/aisuko/datasets-and-dataloaders-in-pytorch). It's time to train, validate and test our model by optimizing its parameters on our data. 

Training a model is an iterative process in each iteration the model makes a guess about the output, calculates the error in its guess(loss), collects the derivatives of the error with respect to its parameters, and **optimizes** these parameters using gradient descent.

# Loading the dataset

We use the dataset from `torchvision`.

In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader=DataLoader(training_data, batch_size=64)
test_dataloader=DataLoader(test_data, batch_size=64)

100%|██████████| 26421880/26421880 [00:01<00:00, 18578695.25it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 306314.55it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5498852.39it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7755846.62it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
len(train_dataloader)

938

# Define the model

In [3]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to('cuda')
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

# Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates.

We define the following hyperparameters for training:

* **Number of Epochs** - the number times to iterate over the dataset.
* **Batch Size** -  the number of data samples propagated through the network before the parameters are updated
* **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [4]:
learning_rate=1e-3
batch_size=64
epochs=5

# Training(Optimization Loop)

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
* **The Train Loop** -  iterate over the training dataset and try to converge to optimal parameters.
* **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.


## Loss Function

When presented with some training data, our untrained network is likely not to give the correct answer. **Loss function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include `nn.MSELoss`(Mean Square Error) for regression tasks, and `nn.NLLLoss`(Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model's output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [5]:
# initialize the loss function
loss_fn=nn.CrossEntropyLoss()

## Optimizer

Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed(here we use Stochastic Gradient Descent). All optimization logic is encapsulated in the `optimizer` object. Here, we use SGD optimizer; additionally, there are many of others avaliable in PyTorch like:
* ADAM
* RMSProp

We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [6]:
import torch
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
* Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up;to prevent double-counting we explicity zero them at each iteration.
* Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
* Once we have our gradients, we call `optimizer.step()` to adjust the paeameters by the gradients collected in the backward pass.

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to('cuda'), y.to('cuda')
        # compute prediction and loss
        pred=model(X)
        loss=loss_fn(pred, y)
        
        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100==0:
            loss,current=loss.item(), batch*batch_size+len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn):
    # set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss, correct=0, 0
    
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during
    # test mode also serves to reduce unnecessary gradient computation and memory usage for
    # tensors with requires_grad=True
    with torch.no_grad():
        for example, label in dataloader:
            example, label = example.to('cuda'), label.to('cuda')
            pred=model(example)
            test_loss+=loss_fn(pred, label).item()
            correct+=(pred.argmax(1)==label).type(torch.float).sum().item()
    test_loss/=num_batches
    correct/=size
    print(f'Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

We initialize the loss function and optimizer, and pass it to `train_loop` and `test_loop`. Feel free to increase the number of epochs to track the model's improving performance.

In [8]:
for t in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

loss: 2.303702 [   64/60000]
loss: 2.280869 [ 6464/60000]
loss: 2.265029 [12864/60000]
loss: 2.264428 [19264/60000]
loss: 2.238822 [25664/60000]
loss: 2.222854 [32064/60000]
loss: 2.232506 [38464/60000]
loss: 2.197511 [44864/60000]
loss: 2.190929 [51264/60000]
loss: 2.156703 [57664/60000]
Test Error: 
 Accuracy:46.9%, Avg loss: 2.146659 

loss: 2.159583 [   64/60000]
loss: 2.141042 [ 6464/60000]
loss: 2.084514 [12864/60000]
loss: 2.105528 [19264/60000]
loss: 2.043950 [25664/60000]
loss: 1.992602 [32064/60000]
loss: 2.021946 [38464/60000]
loss: 1.934820 [44864/60000]
loss: 1.939379 [51264/60000]
loss: 1.869947 [57664/60000]
Test Error: 
 Accuracy:52.3%, Avg loss: 1.857916 

loss: 1.894176 [   64/60000]
loss: 1.855541 [ 6464/60000]
loss: 1.737493 [12864/60000]
loss: 1.787143 [19264/60000]
loss: 1.676717 [25664/60000]
loss: 1.628454 [32064/60000]
loss: 1.659540 [38464/60000]
loss: 1.551335 [44864/60000]
loss: 1.578515 [51264/60000]
loss: 1.481946 [57664/60000]
Test Error: 
 Accuracy:60.1%